In [1]:
import sys
sys.path.append("..")

In [2]:
from solvers.MMR.PQK_solver import *
from circuits.circuits import *
from utils.rbf_kernel_tools import *
from squlearn.encoding_circuit import *


In [3]:
from squlearn.observables import SinglePauli
import numpy as np


def PQK_observable(num_qubits, measurement = "XYZ"):
    """"
    Returns the observable for the PQK solver

    Args:

    num_qubits (int): number of qubits in the system
    measurement (str): measurement operator to be applied to the qubits (default: "XYZ")

    Returns:
    _measurement (list): list of SinglePauli objects representing the measurement operator (shape: num_qubits*len(measurement))

    """
    if isinstance(measurement, str):
                _measurement = []
                for m_str in measurement:
                    if m_str not in ("X", "Y", "Z"):
                        raise ValueError("Unknown measurement operator: {}".format(m_str))
                    for i in range(num_qubits):
                        _measurement.append(SinglePauli(num_qubits, i, op_str=m_str))
    return _measurement

In [4]:
num_qubits = 2
num_features = 1
num_samples = 10
num_layers = 3

sigma = 1
measurement = "XYZ"
_measurement = PQK_observable(num_qubits, measurement)
print(len(_measurement), num_qubits*len(measurement))

6 6


In [5]:
#a = LowLevelQNN(Separable_rx(num_qubits=4, num_layers=1, num_features=2), _measurement, Executor("pennylane"))
#import RBF from sklearn
from sklearn.gaussian_process.kernels import RBF

a = LowLevelQNN(HubregtsenEncodingCircuit(num_qubits=num_qubits, num_layers=num_layers, num_features=num_features), _measurement, Executor("pennylane"))
parameter_array = np.random.rand(a.num_parameters)
print("num_samples", num_samples, "num_features", num_features, "num_parameters", a.num_parameters )
input = np.random.rand(num_samples, num_features)
X = a.evaluate(input, parameter_array, [], "f")["f"] # Shape (num_samples, num_qubits*len(measurement))

num_samples 10 num_features 1 num_parameters 6


In [6]:
O = a.evaluate(input, parameter_array, [], "f")["f"]
dOdx = a.evaluate(input, parameter_array, [], "dfdx")["dfdx"]
dOdxdx = a.evaluate(input, parameter_array, [], "dfdxdx")["dfdxdx"]

In [7]:
dOdx.shape

(10, 6, 1)

In [8]:
dOdxdx.shape

(10, 6, 1, 1)

In [9]:
X.shape

(10, 6)

In [10]:
from utils.rbf_kernel_tools import analytical_derivative_rbf_kernel, analytical_derivative_dot_kernel_2

from utils.rbf_kernel_tools import matrix_rbf, matrix_rbf_dx_slow, matrix_rbf_dxdx_slow

$K(x,y)$

In [11]:
K = RBF(1)(O, O)

Derivative of a kernel 

$\frac{d}{dx} k(x,y) = $

In [12]:
dOdxdx.shape


(10, 6, 1, 1)

In [13]:
K_rbf_dx =  matrix_rbf_dx_slow(X, X, sigma=sigma) #(n,n, d)
Kdx = np.einsum("ijk->ij",(K_rbf_dx*O))

$ \frac{d^{2} K}{dx^2}$

In [14]:
K_rbf_dxdx =  matrix_rbf_dxdx_slow(X, X, sigma=sigma)

Kdxdx = np.zeros((num_samples, num_qubits*len(measurement), num_features, num_features))
Kdxdx_first_term = np.einsum("ijk->ij",(K_rbf_dxdx*dOdx[:,:,0]))
Kdxdx_second_term = np.einsum("ijk->ij",(K_rbf_dx*dOdxdx[:,:,0,0]))
Kdxdx = Kdxdx_first_term + Kdxdx_second_term


In [15]:
Kdxdx_first_term.shape

(10, 10)

Testing implmentation

In [16]:
from circuits.circuits import Separable_rx

In [17]:
sigma = 1
gamma = 1/(2*sigma**2)

experiment = { "sigma": sigma}
experiment["circuit_information"] = {"encoding_circuit": Separable_rx, "num_qubits": 1, "num_layers":1}
pqk =PQK_solver(experiment["circuit_information"],
                                Executor("pennylane"), 
                                envelope={"function": matrix_rbf, 
                                            "derivative_function": matrix_rbf_dx_slow, 
                                            "second_derivative_function": matrix_rbf_dxdx_slow,
                                            "sigma": experiment["sigma"]})

In [47]:
PQK_qnn = pqk.PQK_QNN()
obs_coef = []
X = [0.1, 0.2, 0.3, 0.5, 0.75]
K_f, K_dfdx, K_dfdxdx = pqk.get_PQK_kernel_derivatives(X, PQK_qnn, obs_coef, [1,2])


In [48]:
PQK_qnn

In [49]:
O = PQK_qnn.evaluate(X, [], [], "f")["f"]
dOdx = PQK_qnn.evaluate(X, [], [], "dfdx")["dfdx"]
dOdxdx = PQK_qnn.evaluate(X, [], [], "dfdxdx")["dfdxdx"]

In [21]:
matrix_rbf(O, O, sigma)[0, 1]

0.9950166237049287

In [22]:
O.shape

(5, 3)

In [23]:
O[0]

array([ 0.        , -0.09983342,  0.99500417])

In [24]:
dOdx.shape

(5, 3, 1)

In [25]:
K_dx_manual = np.zeros((len(X), len(X)))
for n in range(len(X)):
   for j in range(len(X)):
      r = 0
      for l in range(O.shape[1]):
         K_dx_manual[n,j] += matrix_rbf_dx_slow(O, O, sigma)[n,j, l] * dOdx[n, l, 0]

In [26]:
# Assuming dOdx has shape (N, L, 1) and the result of matrix_rbf_dx_slow has shape (N, N, L)
# Reshape dOdx to (N, L) to align the dimensions correctly for einsum
dOdx_reshaped = dOdx[:, :, 0]

# Calculate the result using einsum
K_dx_manual = np.einsum('njl, nl->nj', matrix_rbf_dx_slow(O, O, sigma), dOdx_reshaped)


In [27]:
import sympy as sp

x,y,gamma_sp = sp.symbols("x y gamma")

K = sp.exp(-gamma_sp*(2-2*sp.cos(x-y)))
Kx = sp.diff(K, x)
Kxx = sp.diff(Kx, x)

In [64]:
rbf_sp = sp.exp(-gamma_sp*(x-y)**2)

In [66]:
rbf_sp.diff(x, 2)

2*gamma*(2*gamma*(x - y)**2 - 1)*exp(-gamma*(x - y)**2)

In [63]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')
k = sp.symbols('k', cls=sp.Function)
O = sp.symbols('O', cls=sp.Function)
k(O(x1, x2),O(y1, y2)).diff(x1,2)

Derivative(O(x_1, x_2), x_1)**2*Derivative(k(O(x_1, x_2), O(y_1, y_2)), (O(x_1, x_2), 2)) + Derivative(O(x_1, x_2), (x_1, 2))*Derivative(k(O(x_1, x_2), O(y_1, y_2)), O(x_1, x_2))

In [28]:
def K_separable_rx_PQK_(X,Y, gamma):
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))
    return gram_matrix

def K_separable_rx_PQK_dx(X,Y, gamma):
    #-2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)

    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = -2*gamma*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.sin(X[i]-Y[j])
    return gram_matrix 

def K_separable_rx_PQK_dxdx(X,Y, gamma):
    #4*gamma**2*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)**2 - 2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*cos(x - y)
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = 4*gamma**2*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.sin(X[i]-Y[j])**2 - 2*gamma*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.cos(X[i]-Y[j])
    return gram_matrix 

In [29]:
from squlearn.kernel.matrix import ProjectedQuantumKernel

In [30]:
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1), executor=Executor("pennylane"), gamma=gamma)

In [31]:
pqk_squlearn.evaluate(X, X)

array([[1.        , 0.99501662, 0.98026393, 0.92409629, 0.81553071],
       [0.99501662, 1.        , 0.99501662, 0.95631922, 0.8628836 ],
       [0.98026393, 0.99501662, 1.        , 0.98026393, 0.90524206],
       [0.92409629, 0.95631922, 0.98026393, 1.        , 0.96939067],
       [0.81553071, 0.8628836 , 0.90524206, 0.96939067, 1.        ]])

In [32]:
K_separable_rx_PQK_(X,X, gamma)

array([[1.        , 0.99501662, 0.98026393, 0.92409629, 0.81553071],
       [0.99501662, 1.        , 0.99501662, 0.95631922, 0.8628836 ],
       [0.98026393, 0.99501662, 1.        , 0.98026393, 0.90524206],
       [0.92409629, 0.95631922, 0.98026393, 1.        , 0.96939067],
       [0.81553071, 0.8628836 , 0.90524206, 0.96939067, 1.        ]])

In [33]:
K_f

array([[1.        , 0.99501662, 0.98026393, 0.92409629, 0.81553071],
       [0.99501662, 1.        , 0.99501662, 0.95631922, 0.8628836 ],
       [0.98026393, 0.99501662, 1.        , 0.98026393, 0.90524206],
       [0.92409629, 0.95631922, 0.98026393, 1.        , 0.96939067],
       [0.81553071, 0.8628836 , 0.90524206, 0.96939067, 1.        ]])

In [34]:
K_separable_rx_PQK_dx(X,X,gamma)

array([[-0.        ,  0.09933591,  0.19474838,  0.35986004,  0.4935481 ],
       [-0.09933591, -0.        ,  0.09933591,  0.28261165,  0.45101824],
       [-0.19474838, -0.09933591, -0.        ,  0.19474838,  0.3937491 ],
       [-0.35986004, -0.28261165, -0.19474838, -0.        ,  0.23983109],
       [-0.4935481 , -0.45101824, -0.3937491 , -0.23983109, -0.        ]])

In [35]:
K_dfdx

array([[ 0.        ,  0.09933591,  0.19474838,  0.35986004,  0.4935481 ],
       [-0.09933591,  0.        ,  0.09933591,  0.28261165,  0.45101824],
       [-0.19474838, -0.09933591,  0.        ,  0.19474838,  0.3937491 ],
       [-0.35986004, -0.28261165, -0.19474838,  0.        ,  0.23983109],
       [-0.4935481 , -0.45101824, -0.3937491 , -0.23983109,  0.        ]])

In [75]:
gamma

0.5

In [81]:
K_separable_rx_PQK_dxdx(X,X,0.5)

array([[-1.        , -0.98012864, -0.92203339, -0.71101294, -0.35054218],
       [-0.98012864, -1.        , -0.98012864, -0.83008919, -0.49988795],
       [-0.92203339, -0.98012864, -1.        , -0.92203339, -0.64385531],
       [-0.71101294, -0.83008919, -0.92203339, -1.        , -0.8799195 ],
       [-0.35054218, -0.49988795, -0.64385531, -0.8799195 , -1.        ]])

In [37]:
K_dfdxdx

array([[-1.        , -0.98567164, -0.9458699 , -0.81861963, -0.64146701],
       [-0.98539143, -1.        , -0.98644523, -0.89595584, -0.73039335],
       [-0.94308513, -0.98602764, -1.        , -0.95336843, -0.81195068],
       [-0.7921432 , -0.88351708, -0.94948808, -1.        , -0.93858248],
       [-0.54166343, -0.66745102, -0.77811385, -0.93412041, -1.        ]])